In [ ]:
import tensorflow as tf
import numpy as np 

In [ ]:
class LagrangianNN(tf.keras.Model):
    def __init__(self, input_dim, hidden_layers):
        super(LagrangianNN, self).__init__()
        self.hidden_layers = [tf.keras.layers.Dense(units, activation='relu') for units in hidden_layers]
        self.output_layer = tf.keras.layers.Dense(1, activation='linear')

    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
                x = layer(x)
    return self.output_layer(x)

    def lagrangian_loss(model, inputs, outputs, actuator_forces):
    # Predicted Lagrangian
    L_predicted = model(inputs)
    
    # Compute the derivative of L w.r.t inputs
    with tf.GradientTape() as tape:
        tape.watch(inputs)
        L = model(inputs)
    dL_dq = tape.gradient(L, inputs)

    # Compute the loss (mean squared error)
    loss = tf.reduce_mean(tf.square(dL_dq - outputs - actuator_forces))
    return loss




In [ ]:
class HamiltonianNN(tf.keras.Model):
    def __init__(self, input_dim, hidden_layers):
        super(HamiltonianNN, self).__init__()
        self.hidden_layers = [tf.keras.layers.Dense(units, activation='relu') for units in hidden_layers]
        self.output_layer = tf.keras.layers.Dense(1, activation='linear')

    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)